# Tender2Project
The following Kaggle notebook exploits the long context window of Gemini, in order to fulfill the following targets:
- Analyze technical and commercial tenders for a project.
- Analyze the compatibility of products and solution of certain companies with respect to the tender documentations.
- Find the best company and combination of products and services to build the project, generating a clause by clause report with compliant and not-compliant specifications.

## Notebook structure
The notebook is composed by different parts, each one with a specific target:
- Tenders for a project are parsed, such that their information is converted to text.
- Information scraped from the websites of different companies is loaded as text.
- All the text is forwarded to Gemini in different prompts: combining multi-agent reasoning, chain of toughts and in-chat memory.

## Multi-agent reasoning
Multi-agent reasoning is applied in the code through the segmentation of tasks and the delegation of specific responsibilities to distinct roles. For example:

Technical and Commercial Tender Agents: Separate prompts (tender_prompt_template_technical and tender_prompt_template_commercial) are used to guide the technical tender engineer and the commercial tender manager roles, respectively. Each agent has distinct objectives: identifying and summarizing technical or commercial requirements within tenders. This multi-agent structure ensures detailed and domain-specific analyses.

A distinct prompt is also prepared for analyzing companies (e.g., SIEMENS and HITACHI) to match tender requirements with their products and solutions (get_response_companies_info). This allows tailored reasoning for comparing affinity between tenders and company offerings.

## Chain of Thoughts
The chain of thoughts approach is used to decompose complex tasks into sequential, step-by-step actions, ensuring methodical problem-solving. 
In both technical and commercial prompts we used phrases like "Think step by step" to guide the agent toward incremental reasoning. This ensures that requirements are dissected and analyzed in detail.
The user prompt specifies a structured approach to calculating an affinity score, prompting the agent to explicitly explain the calculation process.
Finally in the Clause-by-Clause Analysis, the final prompt directs the agent to meticulously compare tender requirements with company specifications, maintaining a clear progression in thought.
This approach is embedded in the query processing of tenders and the affinity scoring logic in user_prompt_match and final_prompt, encouraging logical progression in the analysis.

## In-chat memory
The code utilizes in-chat memory to maintain conversational context across multiple interactions. This functionality is facilitated by Chat History Preservation: The function add_history_to_chat appends user queries and model responses (e.g., for tenders or company analyses) to history_chat. This ensures continuity, enabling the model to refer back to previous inputs and outputs during subsequent exchanges.
Additionally, prompts such as system_prompt and user_prompt leverage the accumulated chat history to enhance the depth and relevance of responses. For example, when computing affinity scores or performing a clause-by-clause analysis, the model can reference earlier content in the chat_with_memory object. This allows a continous improvement of the prompt and on the information stored in the chat.



## Conclusion for use case
Using a long context window instead of Retrieval-Augmented Generation (RAG) for the selected use case was particularly beneficial due to the nature of the task, which involves reasoning across interdependent documents, maintaining conversational continuity, and ensuring consistent context for decision-making. 
In fact, in the past, we implemented a multi-agent framework using LangChain and OpenAI, where each company was represented by a dedicated agent. The repo is publicily availbla at https://github.com/SecchiAlessandro/LumadaAI. This framework was designed with a supervisor agent that dynamically routed user queries to the most relevant company-specific agent based on the query context. While innovative for that time, this approach encountered several challenges, particularly in stability, accuracy, and efficiency, making the current solution implemented in the notebook a more effective alternative. 
As the agents are operating independently, it was also difficult to generate combined solutions from different companies.
Moreover, for each query, the supervisor had to perform an additional step of reasoning before invoking an agent.
If the query was relevant to multiple agents, the framework had to perform multiple sequential calls, compounding the latency.

The current solution with a centralized reasoning, ensure consistent application of logic and context.
By avoiding the intermediate step of agent selection, it directly processes queries with a unified context, significantly reducing latency.

The unified context allows the model to cross-reference tender requirements and company offerings directly, ensuring a cohesive and accurate analysis.
This is particularly advantageous for tasks like affinity scoring, which require simultaneous consideration of multiple data points.

The notebook’s approach scales better for handling multiple queries simultaneously, as it avoids the bottleneck of sequential agent calls. In fact, for new tender projects, it is just required to update the in-chat memory and to add new prompts for adding new in-chat agents.


In summary, why we decided to implement tender2project?

1. Holistic Context Retention
Long Context Window: By storing the entire history of tender analyses (both technical and commercial) and company product evaluations, the model retains a comprehensive understanding of all previously provided information. This holistic context allows the model to reason about how specific requirements and offerings interrelate across multiple prompts.
In RAG, the system retrieves only the most relevant chunks of information from the documents for each query. While efficient, this approach can fragment the analysis when tasks require synthesizing insights across documents, potentially leading to overlooked interconnections.
2. Interdependent Analysis
Tender and Company Matching: The task involves comparing multiple tenders against products and solutions offered by different companies, followed by calculating an affinity score and conducting a clause-by-clause compliance analysis. These steps require information from previous steps to be accessible and integrated seamlessly.
RAG typically retrieves context independently for each query, which might result in loss of nuance or context-dependent reasoning, especially when relationships between multiple documents must be preserved. A long context window ensures the model has immediate access to the entire conversational flow and insights developed so far.
3. Dynamic Multi-Agent Collaboration
Role-Based Prompts: By maintaining a long context, the system can simulate multi-agent collaboration, where outputs from technical engineers, commercial managers, and sales managers flow into a unified reasoning framework.
In RAG, each role’s analysis would require re-retrieving relevant information from documents, potentially leading to inconsistencies or duplications. With a long context window, outputs from one role naturally inform others, creating a seamless chain of thought.
4. Reduced Query Overhead
Fewer Retrievals, Continuous Focus: Long context windows reduce the need for multiple retrieval calls, making the process more efficient in scenarios where information is revisited or refined iteratively.
RAG introduces latency and computational costs because each query requires searching and ranking document chunks. A long context window allows for continuous focus on the task, with all prior exchanges readily available.
5. Affinity Score Calculation
Global Context for Consistent Metrics: Computing an affinity score across companies for tenders requires integrating technical and commercial analysis alongside company data. This step benefits significantly from the model’s ability to access all previous responses simultaneously.
In RAG, affinity scoring would require separate retrievals of technical requirements, commercial requirements, and company data for each tender. This could introduce discrepancies if context for one query is inadvertently excluded during retrieval.
6. Clause-by-Clause Compliance Analysis
Integrated Insight Application: Clause-by-clause analysis relies on cross-referencing previously extracted requirements with company offerings. The long context window allows the model to directly reference earlier inputs and outputs without reloading or retrieving.
RAG retrievals for clause-by-clause analysis might lead to inconsistencies if prior reasoning is split across multiple retrievals. A long context window ensures the model "remembers" and applies earlier analyses cohesively.
Trade-offs and Use Case Fit

## Conclusion
The centralized, long context window approach provides clear advantages in stability, response time, and accuracy over the earlier multi-agent framework. It highlights the importance of selecting a system architecture that aligns with the specific demands of the use case, particularly for complex, multi-faceted analyses like those in tender evaluations, clause-by-clause generation and company affinity scoring.
#### The long context window acts as a shared workspace, recording and making all agent outputs accessible for seamless and holistic reasoning. In today's interconnected world, where partnerships and synergies are essential to addressing complex challenges, we envision a tool that enables continuous reasoning, uncovers new patterns and solutions, and minimizes the fragmentation of insights.




In [1]:
# import Python libraries
import os
import json
from IPython.display import Markdown

In [2]:
# auxiliary function to read JSON files
def read_json_info(jsonFilePath: str) -> dict:
    if os.path.exists(jsonFilePath):
        with open(jsonFilePath, "r") as f:
            data = json.load(f)
        return data
    else:
        return {}

In [3]:
# auxiliary Python decorator to execute a function again, if its execution fails
# this is helpful when calling the Gemini's API since Gemini has a rate limiter and, if an execution fails for that, there will be some waiting time before retrying
import time

def retry_on_failure(wait_time_seconds=60, max_retries=5):
    def decorator_retry(func):
        
        def wrapper_retry(*args, **kwargs):
            retries = 0
            while retries < max_retries:
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    retries += 1
                    if retries < max_retries:
                        print(
                            f"Function failed with error: {e}. Retrying in {wait_time_seconds} seconds... (Attempt {retries}/{max_retries})"
                        )
                        time.sleep(wait_time_seconds)
                    else:
                        print(f"Function failed after {max_retries} attempts.")
                        raise e
        return wrapper_retry

    return decorator_retry

In [4]:
dataset_path = '/kaggle/input/tenders-and-companies-websites'
working_path = '/kaggle/working'

In [5]:
!mkdir -p /kaggle/working/tenders
tenders_working_path = os.path.join(working_path, 'tenders')

!mkdir -p /kaggle/working/companies
companies_working_path = os.path.join(working_path, 'companies')

# Fetch information about companies

## Overview
Information about interesting companies is obtained from their websites.

To generate data out of the companies' websites, we implemented a crawler.
The final output of the crawler is a JSON file, in which each field refers to a company: for each company, all the information of the websites is merged.

> To make things easier, the mentioned JSON file will be fetched from a Git repository where the crawling function has already been executed.

## Details about the crawling process:
- **Recursive scan**: after a webpage is scanned and its content is stored, eventual found sublinks are scanned, as well. A limit of the wepages to download is given as input.
- **Redundant information is deleted**: if some website content can be found multiple times in all the webpages of one company, then it is skipped. *Example*: undesired and redundant lines like "Contact Us" are removed, ensuring that the final content does not include unnecessary sentences.
- **Caching of already downloaded pages**: for each webpage, the content is stored in a JSON file, as well as the found sublinks. *Example*: after a run with a limit of N pages, other runs with less than N pages will use the stored files instead downloading data from internet; at the contrary, if the limit is increased to M > N pages, only M - N additional pages will be downloaded while the first N pages will be taken from the stored file.

## Load the companies' info

# Chat with Gemini

In [6]:
# API key got here: https://ai.google.dev/tutorials/setup

import google.generativeai as genai
from kaggle_secrets import UserSecretsClient


user_secrets = UserSecretsClient()
secret_key = user_secrets.get_secret("GEMINI_API_KEY")

genai.configure(api_key = secret_key)

model_name = 'gemini-1.5-flash-latest'
model = genai.GenerativeModel(model_name=model_name)

chat = model.start_chat()

model_info = genai.get_model(f"models/{model_name}")
print(f"{model_info.input_token_limit=}")
print(f"{model_info.output_token_limit=}")

model_info.input_token_limit=1000000
model_info.output_token_limit=8192


## Analyze the tenders

In [7]:
# read the json file related to tenders from the input dataset
tenders_info_json_path = os.path.join(dataset_path, 'tenders_info.json')
tenders_info = read_json_info(tenders_info_json_path)

# tenders_info is a dictionary, where the key is the name of the tender file and the related value its information
# print(tenders_info["tender_wind.pdf"])

In [8]:
# list the processed tender files
tenders = tenders_info.keys()

In [9]:
tender_prompt_template_technical = """
You are an experienced technical tender engineer. 
The document you have is a tender, that contains also technical requirements for a project.
Think step by step on how to look for the relevant technical requirements and make a detailed summary.
The content of the document is: """
tender_prompts_technical = []
for info in tenders_info.values():
    tender_prompts_technical.append(f"You have a document called {info['name']} . " + tender_prompt_template_technical + f"{info['content']}")

In [10]:
tender_prompt_template_commercial = """
You are an experienced commercial tender manager. 
The document you have is a tender, that contains also commercial requirements for a project.
Think step by step on how to look for the relevant commercial requirements and make a detailed summary.
The content of the document is: "
"""
tender_prompts_commercial = []
for info in tenders_info.values():
    tender_prompts_commercial.append(f"You have a document called {info['name']} . " + tender_prompt_template_commercial + f"{info['content']}")

In [11]:
@retry_on_failure(wait_time_seconds=60)
def get_responses_tenders(subject, tender_prompts):
    tenders_json_file_path = os.path.join(tenders_working_path, f'tenders_{subject}.json')
    
    if os.path.exists(tenders_json_file_path):
        responses = read_json_info(tenders_json_file_path)
        print(f"tender_{subject}: Responses loaded from file {tenders_json_file_path}")
    else:
        for tender_prompt, tender_name in zip(tender_prompts, tenders):
            print(f"tender_{subject}: Generating response for tender {tender_name} ...")
            response = chat.send_message(tender_prompt)
            # print(response.text)

            responses = {}
            responses[tender_name] = {'prompt': tender_prompt, 'answer': response.text}
            print(f"tender_{subject}: Response for tender {tender_name} generated.")
    
        with open(tenders_json_file_path, 'w') as f:
            json.dump(responses, f, ensure_ascii=True, indent=4)
        print(f"tender_{subject}: Responses stored into {tenders_json_file_path}")
    
    print(f"tender_{subject}: Analysis concluded!\n")
    return responses

# each call of get_responses_tenders() will generate a tenders_{subject}.json file
# each generated file so will contain the Gemnini's responses for a given subject
response_technical = get_responses_tenders("technical", tender_prompts_technical)
response_commercial = get_responses_tenders("commercial", tender_prompts_commercial)

tender_technical: Generating response for tender tender_wind.pdf ...
tender_technical: Response for tender tender_wind.pdf generated.
tender_technical: Generating response for tender tender_solar.pdf ...
tender_technical: Response for tender tender_solar.pdf generated.
tender_technical: Responses stored into /kaggle/working/tenders/tenders_technical.json
tender_technical: Analysis concluded!

tender_commercial: Generating response for tender tender_wind.pdf ...
tender_commercial: Response for tender tender_wind.pdf generated.
tender_commercial: Generating response for tender tender_solar.pdf ...
tender_commercial: Response for tender tender_solar.pdf generated.
tender_commercial: Responses stored into /kaggle/working/tenders/tenders_commercial.json
tender_commercial: Analysis concluded!



## Analyze the companies products and solutions

In [12]:
companies_json_path = os.path.join(dataset_path, 'companies_info.json')
companies_info = read_json_info(companies_json_path)

# companies_info is a dictionary, where the key is the name of the company and the related value its information
# print(companies_info["SIEMENS"]) 

In [13]:
@retry_on_failure(wait_time_seconds=60)
def get_response_companies(company_name):
    companies_json_file_path = os.path.join(companies_working_path, f'companies_{company_name}.json')
    
    if os.path.exists(companies_json_file_path):
        responses = read_json_info(companies_json_file_path)
        print(f"companies_{company_name}: Responses loaded from file {companies_json_file_path}")
    else:
        print(f"companies_{company_name}: Generating response for company {company_name} ...")
        responses = {}
        company_prompt = f"These are the information of products and solutions for the company {company_name} : {companies_info[company_name]}"
        response = chat.send_message(company_prompt)
        responses[company_name] = {'prompt': company_prompt, 'answer': response.text}

        with open(companies_json_file_path, 'w') as f:
                json.dump(responses, f, ensure_ascii=True, indent=4)
        print(f"companies_{company_name}: Responses stored into {companies_json_file_path}")

    print(f"companies_{company_name}: Response for company {company_name} generated!")
    return responses

# each call of get_responses_companies() will generate a companies_{company_name}.json file
# each generated file so will contain the Gemnini's responses for a given company
# the purpose of generating responses given companies information is to store it in the chat history
response_hitachi = get_response_companies("HITACHI")
response_siemens = get_response_companies("SIEMENS")

companies_HITACHI: Generating response for company HITACHI ...
companies_HITACHI: Responses stored into /kaggle/working/companies/companies_HITACHI.json
companies_HITACHI: Response for company HITACHI generated!
companies_SIEMENS: Generating response for company SIEMENS ...
Function failed with error: 429 Resource has been exhausted (e.g. check quota).. Retrying in 60 seconds... (Attempt 1/5)
companies_SIEMENS: Generating response for company SIEMENS ...
companies_SIEMENS: Responses stored into /kaggle/working/companies/companies_SIEMENS.json
companies_SIEMENS: Response for company SIEMENS generated!


## Build a chat history based on previous prompts

In [14]:
# example how to include the chat history here https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_chat.ipynb
# description of the Content class here https://github.com/google-gemini/generative-ai-python/blob/main/docs/api/google/generativeai/GenerativeModel.md
from google.generativeai.protos import Content, Part

history_chat = []

def add_history_to_chat_single(response, user, history_chat):
    query = Part()
    query.text = f"{user}: {response['prompt']}"
    history_chat.append(Content(role="user", parts=[query]))

    answer = Part()
    answer.text = response['answer']
    history_chat.append(Content(role="model", parts=[answer]))
    return

def add_history_to_chat(responses, user, history_chat):
    for response in responses.values():
        add_history_to_chat_single(response, user, history_chat)
    return 

add_history_to_chat(response_technical, "technical engineer", history_chat)
add_history_to_chat(response_commercial, "commercial manager", history_chat)
add_history_to_chat(response_siemens, "sales manager for siemens", history_chat)
add_history_to_chat(response_hitachi, "sales manager for hitachi", history_chat)

## Test the chat history

In [15]:
# the decorator ensures that, if an error occurs, the function will be executed again
@retry_on_failure(wait_time_seconds=60, max_retries=3)
def ask_gemini_with_history(prompt, chat_with_memory=None, model=None, history=[]):
    """
    function to call Gemini, providing chat history
    if a chat is already available, it will be used
    """
    if chat_with_memory == None:
        # since no chat is already available, create a new one
        chat_with_memory = model.start_chat(history=history)
    
    response = chat_with_memory.send_message(prompt)
    return response, chat_with_memory

In [16]:
prompt_roles = "Which are the roles given in the prompt from the user? There are only two for tenders and one for company"
response_roles, gemini_chat = ask_gemini_with_history(prompt=prompt_roles, model=model, history=history_chat)

Markdown(response_roles.text)

The prompt assigns three distinct roles:

1. **Technical Tender Engineer:** This role focuses on the technical aspects of a tender document, specifically identifying and summarizing the technical requirements of a project.

2. **Commercial Tender Manager:** This role centers on the commercial aspects of a tender, identifying and summarizing the financial and contractual requirements.

3. **Sales Manager (for Siemens/Hitachi):** This is a company-specific role focused on understanding the company's product portfolio and how it aligns with the needs outlined in a tender document.  The prompt requests analysis of each company's offerings, not specific roles *within* the company.


In [17]:
# add the last response to the chat history
add_history_to_chat_single({'prompt': prompt_roles, 'answer': response_roles.text}, "technical engineer", history_chat)

## Find the most suitable company

In [18]:
prompt_match = """

1. For company SIEMENS and HITACHI, find the respective relevant products and solutions with respect to the analyzed tenders.
   The information is in the form of text I provided, then you do not need to read additional documents or access to websites.
   
   
2. Calculate an affinity score in percentage for each company based on analysis in point 1. Explain the way how you computed this percentage.

"""

In [19]:
print("Finding the most suitable company for the tenders ...")
response_match, gemini_chat = ask_gemini_with_history(prompt=prompt_match, chat_with_memory=gemini_chat)
print("Response to the prompts is ready!")

Markdown(response_match.text)

Finding the most suitable company for the tenders ...
Function failed with error: 429 Resource has been exhausted (e.g. check quota).. Retrying in 60 seconds... (Attempt 1/3)
Response to the prompts is ready!


To perform this analysis, I need the tender specifications.  The previous responses analyzed the Siemens and Hitachi product information but lacked the crucial information: the detailed requirements of the Barclayville Solar Power Plant tender.  Please provide the complete tender specifications so I can match them against the companies' offerings and calculate affinity scores.


Once you provide the tender specifications, my analysis will proceed in these steps:

**Step 1: Identifying Relevant Products and Solutions**

I will compare each item in the tender document against the Siemens and Hitachi product descriptions.  For each tender item, I will note which products or solutions from each company are relevant.  This will require a detailed comparison and might include multiple products from one company to satisfy a single tender item.

**Step 2: Calculating Affinity Scores**

The affinity score will be a percentage representing the degree to which each company's offerings match the tender's requirements.  I will calculate this in a weighted fashion:

* **Weighting of Tender Items:**  First, I will assign weights to each item in the tender based on its relative importance (as implied by the tender document or through additional information you can provide).  For instance, the solar array might be weighted higher than the spare parts. The more important the tender item, the higher the weight.  The sum of all weights should equal 1 (or 100%).

* **Match Score for Each Item:** For each tender item, I will assign a match score (0-1 or 0-100%) to each company's relevant offering.  A score of 1 (or 100%) indicates a perfect match; 0 (or 0%) means no match. Partial matches will receive intermediate scores.  The match score reflects both the technical and commercial suitability of the offering.

* **Weighted Affinity Score:**  The weighted affinity score for each company will be calculated as the sum of (Weight of Tender Item * Match Score for that item) across all tender items. This will be expressed as a percentage.

**Example:**

Let's say the tender has three items: Solar Array (Weight 0.5), Inverter (Weight 0.3), and Monitoring System (Weight 0.2).

| Company | Solar Array Match | Inverter Match | Monitoring System Match | Weighted Affinity Score |
|---|---|---|---|---|
| Siemens | 0.9 (90%) | 0.8 (80%) | 1.0 (100%) | (0.5 * 0.9) + (0.3 * 0.8) + (0.2 * 1.0) = 0.83 or 83% |
| Hitachi | 0.7 (70%) | 0.9 (90%) | 0.7 (70%) | (0.5 * 0.7) + (0.3 * 0.9) + (0.2 * 0.7) = 0.7 or 70% |


This example shows a higher affinity score for Siemens. The final scores will depend heavily on the specific tender requirements and the weighting assigned to each item.  Provide the tender details for a complete analysis.


In [20]:
# add the last response to the chat history
add_history_to_chat_single({'prompt': prompt_match, 'answer': response_match.text}, "technical engineer", history_chat)

## Generate the clause-by-clause

In [21]:
user_prompt = """

Consider the company with the highest affinity score 
and return the clause by clause analysis considering technical and commercial compliant and not-compliant requirements
of the tender with respect to the selected company. Report also the URL of the source where you found the informations.

"""

In [22]:
system_prompt = """
You are an experienced team of business development managers and tender engineers, commercial managers.
You need to create a detailed clause by clause from the tender documentations and the most affine company specifications.
"""

In [23]:
print("Generating the clause by clause ...")
prompt_clause_by_clause = f"{system_prompt} {user_prompt}"
response_clause_by_clause, gemini_chat = ask_gemini_with_history(prompt=prompt_clause_by_clause, chat_with_memory=gemini_chat)
print("Response to the prompts is ready!")

Markdown(response_clause_by_clause.text)

Generating the clause by clause ...
Function failed with error: 429 Resource has been exhausted (e.g. check quota).. Retrying in 60 seconds... (Attempt 1/3)
Response to the prompts is ready!


I cannot provide a clause-by-clause analysis of a tender document and company specifications without the actual tender document and the relevant company specifications. My previous responses emphasized that I need this information to proceed.

To create the requested analysis, please provide:

1. **The complete tender document for the Barclayville Solar Power Plant:** This should include all clauses, technical specifications, and commercial requirements.

2. **The company specifications (relevant product descriptions and capabilities):**  This should be the detailed technical and commercial information for the company with the highest affinity score (as determined by the analysis in the previous response, once you provide the tender). Ideally, you will provide links to the relevant source documents to verify accuracy, as I cannot access external websites.

Once you provide these documents, I can conduct a detailed, clause-by-clause analysis comparing the tender requirements with the selected company's capabilities. The analysis will indicate which clauses are met, which are not, and the reasons for compliance or non-compliance (both technically and commercially). This will involve:

* **Careful Review:**  Reading each clause in the tender document to understand the requirements.

* **Matching to Company Specifications:**  Identifying the corresponding company specifications or capabilities that would satisfy each clause.

* **Compliance Assessment:**  Determining whether the company’s offerings meet each clause's technical and commercial requirements. This will necessitate precise, detailed comparison of specifications.

* **Non-Compliance Reasons:**  Explaining why a clause is not met (e.g., missing features, different specifications, cost implications).

* **Structured Reporting:**  Presenting the analysis in a clear, well-organized format (likely a table), with each clause addressed and the relevant company information and compliance status provided.  Each line item will include the URL of the source document.


Without the required documents, I cannot perform the requested clause-by-clause analysis.


In [24]:
# add the last response to the chat history
add_history_to_chat_single({'prompt': prompt_clause_by_clause, 'answer': response_clause_by_clause.text}, "technical engineer", history_chat)

## Count the overall tokens

The total number of token can be computed by counting the tokens of history_chat, since the new responses have been appended to it for each call of Gemini.

In [25]:
print(f"{model.count_tokens(history_chat)=}")

model.count_tokens(history_chat)=total_tokens: 664426

